In [2]:
# Enter your OpenAI API key here:
# import os
# os.environ['OPENAI_API_KEY'] = ''

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains.sequential import SequentialChain

In [2]:
# Setup the chat model:
chat = ChatOpenAI()

In [3]:
# Create the article outline generation chain:
template = """You are responsible for creating a very detailed article outline for {blog_post_idea}"""
article_outline_generation_chain = LLMChain(
    llm=chat,
    prompt=ChatPromptTemplate.from_template(template),
    output_key="article_outline",
)

In [4]:
# Create the article generation chain:
template = """Given an article outline, write a very high quality and detailed article.
    The article must be at least {word_count} words in length.
    Article outline: {article_outline}
    """
article_generation_chain = LLMChain(
    llm=chat,
    prompt=ChatPromptTemplate.from_template(template),
    output_key="generated_article",
)

In [5]:
# Create the article editing chain:
template = """Given an article, edit it to be very high quality and detailed.
    The article must be at least {word_count} words in length.

    You must follow the following principles:
    1. The article must be very detailed and high quality.
    2. If the article is under {word_count} words, you must add more content to it.
    3. If you see any grammatical errors, you must fix them.
    4. If you see any spelling errors, you must fix them.
    5. If you see any factual errors, you must fix them.

    Here is the article: {generated_article}
    """
article_editing_chain = LLMChain(
    llm=chat, prompt=ChatPromptTemplate.from_template(template), output_key="edited_article"
)

In [6]:
# Combine three chains into a single chain:
chain = SequentialChain(
    chains=[
        article_outline_generation_chain,
        article_generation_chain,
        article_editing_chain,
    ],
    input_variables=["blog_post_idea", "word_count"],
    output_variables=["article_outline", "generated_article", "edited_article"],
    verbose=True,
)

In [7]:
# Run the chain:
article_results = chain(
    {
        "blog_post_idea": "How to become a better data engineer.",
        "word_count": 1000,
    },
)



> Entering new SequentialChain chain...


In [ ]:
article_results["edited_article"]

"Introduction\n\nIn today's digital era, the role of a data engineer has become increasingly important. With businesses relying on data to gain insights, improve efficiency, and drive innovation, data engineering plays a crucial role in managing and processing the vast amounts of data generated daily. In this article, we will provide a detailed overview of the role of a data engineer, covering their responsibilities, required skills, and the intersection between data engineering and other data-related roles. We will also delve into various aspects of data engineering, including data modeling and database design, data integration and ETL processes, data warehousing and data lakes, data pipeline reliability and scalability, data governance and security, and the importance of continuous learning and professional development in this field.\n\nUnderstanding the Role of a Data Engineer\n\nA data engineer is a professional responsible for designing, building, and maintaining the systems and i

# Translating The Article To English:


In [ ]:
from __future__ import annotations
from typing import Any, Dict, List, Optional

from pydantic import Extra
from langchain.callbacks.manager import (
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class MyCustomChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: Any
    output_key: str = "text"  #: :meta private:
    languages: List[str] = ["French", "Spanish", "German"]  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # The custom chain logic goes here:
        translations = {}
        for language in self.languages:
            prompt_value = self.prompt.format_prompt(
                **{"language": language, "edited_article": inputs["edited_article"]}
            )
            translations[language] = self.llm.generate_prompt(
                [prompt_value],
                callbacks=run_manager.get_child() if run_manager else None,
            )

        return {self.output_key: translations}

    @property
    def _chain_type(self) -> str:
        return "my_custom_chain"

In [ ]:
chain = MyCustomChain(
    prompt=ChatPromptTemplate.from_template(
        """You are receiving an article and must translate it into {language}.\n\n{edited_article}"""
    ),
    llm=ChatOpenAI(),
    output_key="translations",
)

result = chain({"edited_article": "I love learning programming!", "language": "French"})

In [ ]:
print(result["translations"]["French"].generations[0][0].text)
print(result["translations"]["Spanish"].generations[0][0].text)
print(result["translations"]["German"].generations[0][0].text)

J'adore apprendre la programmation !
¡Me encanta aprender programación!
Ich liebe es, Programmieren zu lernen!


---

Now you can easily add the custom translation chain to your previous `SequentialChain` object:


In [ ]:
translation_sequential_chain = SequentialChain(
    chains=[
        article_outline_generation_chain,
        article_generation_chain,
        article_editing_chain,
        chain,
    ],
    input_variables=["blog_post_idea", "word_count", "language"],
    output_variables=[
        "article_outline",
        "generated_article",
        "edited_article",
        "translations",
    ],
    verbose=True,
)

In [ ]:
translation_chain_results = translation_sequential_chain(
    {
        "blog_post_idea": "How to become a better data engineer.",
        "word_count": 1000,
        "language": "",  # Dummy key here as you are accessing the languages via self.languages on the custom LLM chain:
    },
)



> Entering new SequentialChain chain...

> Finished chain.


In [ ]:
translation_chain_results

{'blog_post_idea': 'How to become a better data engineer.',
 'word_count': 1000,
 'language': '',
 'article_outline': 'I. Introduction\n    A. Definition of a data engineer\n    B. Importance of data engineering in the current technological landscape\n    C. Purpose of the article – to provide a comprehensive guide on becoming a better data engineer\n\nII. Understanding the Role of a Data Engineer\n    A. Duties and responsibilities of a data engineer\n    B. Key skills and knowledge required\n    C. Importance of continuous learning and adaptability in the field\n\nIII. Building a Strong Foundation\n    A. Education and formal qualifications\n    B. Recommended academic disciplines and degrees\n    C. Importance of understanding basic concepts in computer science and programming\n\nIV. Essential Technical Skills for Data Engineers\n    A. Proficiency in programming languages (Python, Java, etc.)\n    B. Data manipulation and querying (SQL, NoSQL, etc.)\n    C. Big data technologies (H